In [1]:
# use conda enviroment "general" to run this script

import pysam
import glob
import csv
import gzip
import os

from extract_barcode_from_each_read import extract_barcode_from_each_read
from write_reads_from_BAM_file_to_FASTQ_gz_file import write_reads_from_BAM_file_to_FASTQ_gz_file

In [8]:
file_directory = '/Volumes/lab-frattap/home/users/jeanz/Nanopore_sequencing/ELK1_UG_replaced_library_in_Dox_curve/'

all_BAM_files = glob.glob(file_directory + '*.bam')

In [9]:
# read the library of each barcode and its corresponding insert in
barcode_insert_library = '/Users/jeanz/GitHub/replace_UG_in_first_800_bp_of_ELK1_extended_3_prime_UTR/8_get each insert corresponding barcodes by Nanopore sequencing 10_P_1 library/barcode_insert_library.csv.gz'

barcode_dict = {}

with gzip.open(barcode_insert_library, mode='rt') as CSV_file:
    
    csv_reader = csv.reader(CSV_file)
    
    next(csv_reader) # skip the header
    
    for row in csv_reader:
        
        # row[0]: rname
        # row[1]: barcode
        
        read_counts = int(row[2]) # int(): convert string to integer
        
        if read_counts >= 1:
            
            barcode_dict[row[1]] = row[0]
        
#print(barcode_dict)

In [10]:
# 15-mer barcode start and end positions in the reference
barcode_start_in_Ref = 724
barcode_end_in_Ref = 738

In [14]:
def main(BAM_file):
    
    dict = {}
    
    with pysam.AlignmentFile(BAM_file) as bam:
                
        for read in bam:
            
            if read.is_supplementary or read.is_secondary or read.is_unmapped:
                
                continue
            
            #if read.mapping_quality < 20:
                
            #    continue
            
            
            
            query_name = read.query_name
            query_sequence = read.query_sequence
            alignment = read.get_aligned_pairs()
            
            
            
            barcode = extract_barcode_from_each_read(query_sequence, alignment, barcode_start_in_Ref, barcode_end_in_Ref)
            
            if barcode not in barcode_dict.keys():
                
                continue
            
            expected_reference = barcode_dict[barcode]
            
            
            
            if expected_reference not in dict.keys():
                
                dict[expected_reference] = [read]
            
            else:
                
                dict[expected_reference] += [read]




    
    for expected_reference, reads in dict.items():
        
        # extract the Nanopore barcode from the BAM file name
        file_name = BAM_file.split('/')[-1]
        Nanopore_barcode = file_name.split('.')[0]
        
        
        
        output_path = '/Volumes/lab-frattap/home/users/jeanz/Nanopore_sequencing/ELK1_UG_replaced_library_in_Dox_curve/splitted_FASTQ_files/'
        os.makedirs(output_path, exist_ok=True)  # exist_ok=True will prevent errors if the directory already exists
        
        
        
        FASTQ_gz_file = output_path + Nanopore_barcode + '_____' + expected_reference + '.fastq.gz'
        
        write_reads_from_BAM_file_to_FASTQ_gz_file(FASTQ_gz_file, reads)

In [20]:
for BAM_file in all_BAM_files:
    
    print ("start processing", BAM_file)
    
    main(BAM_file)

start processing /Volumes/lab-frattap/home/users/jeanz/Nanopore_sequencing/ELK1_UG_replaced_library_in_Dox_curve/barcode02.sorted.bam
start processing /Volumes/lab-frattap/home/users/jeanz/Nanopore_sequencing/ELK1_UG_replaced_library_in_Dox_curve/barcode05.sorted.bam
start processing /Volumes/lab-frattap/home/users/jeanz/Nanopore_sequencing/ELK1_UG_replaced_library_in_Dox_curve/barcode01.sorted.bam
start processing /Volumes/lab-frattap/home/users/jeanz/Nanopore_sequencing/ELK1_UG_replaced_library_in_Dox_curve/barcode08.sorted.bam
start processing /Volumes/lab-frattap/home/users/jeanz/Nanopore_sequencing/ELK1_UG_replaced_library_in_Dox_curve/barcode10.sorted.bam
start processing /Volumes/lab-frattap/home/users/jeanz/Nanopore_sequencing/ELK1_UG_replaced_library_in_Dox_curve/barcode04.sorted.bam
start processing /Volumes/lab-frattap/home/users/jeanz/Nanopore_sequencing/ELK1_UG_replaced_library_in_Dox_curve/barcode09.sorted.bam
start processing /Volumes/lab-frattap/home/users/jeanz/Nanopor